In [8]:
import time
from data_crawler import data_crawler

data_crawler = data_crawler(None)

ticker = "BTCUSDT_UMCBL"
now = int(time.time() * 1000)
supported_period = data_crawler.get_supported_period()

downloaded_datas = dict()

for period in supported_period.keys():
    downloaded_datas[period] = data_crawler.download_datas(ticker=ticker,
                                                           period=period, 
                                                           startTime=now - supported_period[period][0] * supported_period[period][1],
                                                           endTime=now)

print(downloaded_datas)

Succeed to download datas [BTCUSDT_UMCBL / 5m / 1695931944201 / 1695961944201]
Succeed to download datas [BTCUSDT_UMCBL / 15m / 1695871944304 / 1695961944304]
Succeed to download datas [BTCUSDT_UMCBL / 1H / 1695601944403 / 1695961944403]
Succeed to download datas [BTCUSDT_UMCBL / 6H / 1693801944496 / 1695961944496]
Succeed to download datas [BTCUSDT_UMCBL / 1D / 1687321944595 / 1695961944595]
Succeed to download datas [BTCUSDT_UMCBL / 5m / 1695931944692 / 1695961944692]
Succeed to download datas [BTCUSDT_UMCBL / 15m / 1695871944692 / 1695961944692]
Succeed to download datas [BTCUSDT_UMCBL / 1H / 1695601944692 / 1695961944692]
Succeed to download datas [BTCUSDT_UMCBL / 6H / 1693801944692 / 1695961944692]
Succeed to download datas [BTCUSDT_UMCBL / 1D / 1687321944692 / 1695961944692]
{'5m': b'[["1695931800000","27152.9","27154.5","27092.8","27104.1","303.329","8222531.0717"],["1695932100000","27104.1","27151","27096.1","27150.7","148.436","4025943.6753"],["1695932400000","27150.7","27150.